In [1]:
import pandas as pd
import json
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


%matplotlib inline

with open('/home/monica/leads_leadsq.json') as data_file:
   data = json.load(data_file)


/home/monica/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
mp_data = pd.read_csv('/home/monica/iPythonNotebooks/mixpanel_leads_29_sep_2018.csv')

/home/monica/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
num_records = len(data['Leads'])
print(num_records)

num_columns = len(data['Leads'][0]['LeadPropertyList'])

lead_properties = data['Leads'][0]['LeadPropertyList'][0:num_columns]

column_names = [lead_properties[column]['Attribute'] for column in range(0,num_columns)]

print("Column names for the leads.")
print("=" * 100)
print(column_names)

data_lead = [[data['Leads'][record]['LeadPropertyList'][column]['Value'] for column in range(num_columns)] for record in range(num_records)]

lead_df = pd.DataFrame(columns = column_names, data = data_lead)

lead_df.rename(columns={'EmailAddress': 'distinct_id'}, inplace=True)

print(lead_df.head())

print(lead_df.info())



97240
Column names for the leads.
['ProspectActivityDate_Min', 'Web_Referrer', 'Web_RefKeyword', 'LeadLastModifiedOn', 'ConversionReferrerURL', 'SourceReferrerURL', 'SourceIPAddress', 'Latitude', 'Longitude', 'ProspectID', 'ProspectAutoId', 'ProspectActivityId_Max', 'ProspectActivityId_Min', 'StatusCode', 'StatusReason', 'DeletionStatusCode', 'RelatedProspectId', 'SourceReferrer', 'NotableEvent', 'NotableEventdate', 'LastVisitDate', 'RelatedLandingPageId', 'FirstLandingPageSubmissionId', 'FirstLandingPageSubmissionDate', 'ProspectActivityName_Max', 'ProspectActivityDate_Max', 'FirstName', 'LastName', 'EmailAddress', 'Phone', 'Company', 'Website', 'DoNotTrack', 'DoNotEmail', 'DoNotCall', 'Source', 'Mobile', 'SourceCampaign', 'SourceMedium', 'JobTitle', 'SourceContent', 'Notes', 'Score', 'EngagementScore', 'Revenue', 'ProspectStage', 'QualityScore01', 'OwnerId', 'CreatedBy', 'CreatedByName', 'CreatedOn', 'LeadConversionDate', 'ModifiedBy', 'ModifiedByName', 'ModifiedOn', 'TimeZone', 'Tot

In [4]:
print(mp_data.info())
common_data = pd.merge(lead_df, mp_data, on='distinct_id')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32914 entries, 0 to 32913
Data columns (total 28 columns):
Unnamed: 0                  32914 non-null int64
id                          32914 non-null object
event                       32914 non-null object
utm_content                 75 non-null object
browser_version             20399 non-null float64
search_engine               8866 non-null object
city                        19271 non-null object
initial_referring_domain    20544 non-null object
lib_version                 20544 non-null object
utm_medium                  7504 non-null object
browser                     20544 non-null object
screen_width                20544 non-null float64
initial_referrer            20544 non-null object
utm_campaign                7467 non-null object
time                        32914 non-null int64
current_url                 20544 non-null object
distinct_id                 32914 non-null object
utm_source                  7555 non-null objec

In [5]:
print(common_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23827 entries, 0 to 23826
Columns: 159 entries, ProspectActivityDate_Min to is_valid_email
dtypes: bool(1), float64(3), int64(2), object(153)
memory usage: 28.9+ MB
None


In [6]:
common_data.to_csv('/home/monica/l_m_data.csv')

In [7]:
with open('/home/monica/ld_act.json') as data_file:
   act_hist_data = json.load(data_file)


In [10]:
for i in range(100):
    print(act_hist_data[i])

{'_id': {'$oid': '5b60209723f18b6a5a344e97'}, 'ProspectActivityId': 'dc88b9fd-3841-4553-9b9a-0537a9c03cc7', 'RelatedProspectId': 'e9708477-335c-44e7-8930-1a7cbe97cf2c', 'ActivityEvent': '179', 'ActivityEventName': 'CallBack-Scheduled', 'ActivityType': '2', 'Score': 1, 'CreatedBy': 'df2c81ac-529f-11e8-8401-22000aa79843', 'CreatedOn': '2018-07-23 14:15:00', 'Data': {'Note': 'rnr', 'Status': '', 'Owner': '', 'mx_Custom_1': '2018-07-24 14:15:00', 'mx_Custom_2': 'Engaged Lead'}}
{'_id': {'$oid': '5b60209723f18b6a5a344e98'}, 'ProspectActivityId': 'ad5ebb99-ff4d-4c8e-b968-d297ba1725cc', 'RelatedProspectId': '663c2a81-68df-11e8-8401-22000aa79843', 'ActivityEvent': '179', 'ActivityEventName': 'CallBack-Scheduled', 'ActivityType': '2', 'Score': 1, 'CreatedBy': 'df2c81ac-529f-11e8-8401-22000aa79843', 'CreatedOn': '2018-07-23 14:18:00', 'Data': {'Note': 'rnr', 'Status': '', 'Owner': '', 'mx_Custom_1': '2018-07-24 14:18:00', 'mx_Custom_2': 'Engaged Lead'}}
{'_id': {'$oid': '5b60209823f18b6a5a344e99